## 1. Load packages

In [23]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [24]:
import torch
import numpy as np
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer, Trainer, AutoModelForSequenceClassification, BertForSequenceClassification, BertConfig,BertModel
from datasets import load_dataset, ClassLabel, Value, load_metric

## 2.Prepare dataset for finetuning

In [25]:
# load the tokenizer pretrained on bert-base
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at C:\Users\52673/.cache\huggingface\transformers\886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "result/bert-base-uncased-cls_before_pooler-sym_mlp-mlp_bert-bs64-gpu8-gs1-lr5e-5-m=stsb-norm0.05-l32-contra",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  

In [26]:
# sampling the dataset for fine-tuning
train = load_dataset("quora", split = 'train[:35%]')     # classic 7/3 split
validation = load_dataset("quora", split = 'train[35%:50%]')

Using custom data configuration default
Reusing dataset quora (C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)
Using custom data configuration default
Reusing dataset quora (C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)


In [27]:
# convert the dtype of 'is_duplicate' to int
new_features = train.features.copy()
new_features["is_duplicate"] = Value('int32')
train = train.cast(new_features)

new_features = validation.features.copy()
new_features["is_duplicate"] = Value('int32')
validation = validation.cast(new_features)

Loading cached processed dataset at C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04\cache-0474a84a76505170.arrow
Loading cached processed dataset at C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04\cache-387810a1332fde55.arrow


In [28]:
# encode the training dataset in the form of sentences pair
# truncate at length=64 for a balance of time consuming and information coverage
encoded_train = train.map(lambda batch: tokenizer(batch['questions']['text'][0], batch['questions']['text'][1], 
                                                  padding='max_length', truncation=True, max_length=64))
encoded_train.rename_column_("is_duplicate", "labels")

Loading cached processed dataset at C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04\cache-94540c3e4c186fdb.arrow


In [29]:
# encode the validation dataset in the form of sentences pair
encoded_validation = validation.map(lambda batch: tokenizer(batch['questions']['text'][0], batch['questions']['text'][1], 
                                                            padding='max_length', truncation=True, max_length=64))
encoded_validation.rename_column_("is_duplicate", "labels")

100%|██████████| 60643/60643 [00:31<00:00, 1950.24ex/s]


In [30]:
encoded_train.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
encoded_validation.set_format("torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

In [31]:
# check if the format is valid, and if each tensor have the same size
print({key: val.shape for key, val in encoded_validation[0].items()})
print({key: val.shape for key, val in encoded_validation[1].items()})

{'labels': torch.Size([]), 'input_ids': torch.Size([64]), 'token_type_ids': torch.Size([64]), 'attention_mask': torch.Size([64])}
{'labels': torch.Size([]), 'input_ids': torch.Size([64]), 'token_type_ids': torch.Size([64]), 'attention_mask': torch.Size([64])}


In [32]:
# take a look at an encoded sample
encoded_train[1]

{'labels': tensor(0),
 'input_ids': tensor([  101,  2054,  2003,  1996,  2466,  1997, 12849, 10606, 16506,  1006,
         12849,  2232,  1011,  1045,  1011,  2053,  2953,  1007,  6323,  1029,
           102,  2054,  2052,  4148,  2065,  1996,  2796,  2231, 10312,  1996,
         12849, 10606, 16506,  1006, 12849,  2232,  1011,  1045,  1011,  2053,
          2953,  1007,  6323,  2067,  1029,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 

## 3. Fine-tune

In [33]:
# another way to load pretrained weights
output_config_file = "./config.json"
output_model_file = "./sup-simcse.bin"

config = BertConfig.from_json_file(output_config_file)
model = BertModel(config = config)
state_dict = torch.load(output_model_file)
model.load_state_dict(state_dict)
pass

In [34]:
# load pretrained weights to initialize Bert
model = BertForSequenceClassification.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at C:\Users\52673/.cache\huggingface\transformers\886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "result/bert-base-uncased-cls_before_pooler-sym_mlp-mlp_bert-bs64-gpu8-gs1-lr5e-5-m=stsb-norm0.05-l32-contra",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "use_cache": true,
  

In [13]:
# clean up gpu cache before training
import gc

gc.collect()

torch.cuda.empty_cache()

In [14]:
# train
trainer = Trainer(model=model, train_dataset=encoded_train, eval_dataset=encoded_validation)

trainer.train()    
#trainer.train(resume_from_checkpoint=True) # True if already trained, to save time by continuing on a checkpoint

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: questions.
***** Running training *****
  Num examples = 141502
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 53064
  1%|          | 500/53064 [01:42<2:57:48,  4.93it/s]Saving model checkpoint to tmp_trainer\checkpoint-500
Configuration saved in tmp_trainer\checkpoint-500\config.json


{'loss': 0.6375, 'learning_rate': 4.9528870797527514e-05, 'epoch': 0.03}


Model weights saved in tmp_trainer\checkpoint-500\pytorch_model.bin
  2%|▏         | 1000/53064 [03:26<2:54:51,  4.96it/s]Saving model checkpoint to tmp_trainer\checkpoint-1000
Configuration saved in tmp_trainer\checkpoint-1000\config.json


{'loss': 0.6511, 'learning_rate': 4.9057741595055026e-05, 'epoch': 0.06}


Model weights saved in tmp_trainer\checkpoint-1000\pytorch_model.bin
  3%|▎         | 1500/53064 [05:06<2:43:17,  5.26it/s]Saving model checkpoint to tmp_trainer\checkpoint-1500
Configuration saved in tmp_trainer\checkpoint-1500\config.json


...

Model weights saved in tmp_trainer\checkpoint-52500\pytorch_model.bin
100%|█████████▉| 53000/53064 [4:42:22<00:21,  2.98it/s]Saving model checkpoint to tmp_trainer\checkpoint-53000
Configuration saved in tmp_trainer\checkpoint-53000\config.json


{'loss': 0.6524, 'learning_rate': 6.030453791647821e-08, 'epoch': 3.0}


Model weights saved in tmp_trainer\checkpoint-53000\pytorch_model.bin
100%|██████████| 53064/53064 [4:42:47<00:00,  2.98it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 53064/53064 [4:42:47<00:00,  3.13it/s]

{'train_runtime': 16967.8019, 'train_samples_per_second': 25.018, 'train_steps_per_second': 3.127, 'train_loss': 0.6604022006364533, 'epoch': 3.0}


TrainOutput(global_step=53064, training_loss=0.6604022006364533, metrics={'train_runtime': 16967.8019, 'train_samples_per_second': 25.018, 'train_steps_per_second': 3.127, 'train_loss': 0.6604022006364533, 'epoch': 3.0})

In [15]:
# save the fine_tuned model
model.save_pretrained("simcse_qqp")

Configuration saved in simcse_qqp\config.json
Model weights saved in simcse_qqp\pytorch_model.bin


# Evaluation

In [13]:
# sampling the evaluation dataset from QQP
train_eval = load_dataset("quora", split = 'train[50%:85%]')
validation_eval = load_dataset("quora", split = 'train[85%:100%]')

# repeat all the preprocessing steps above
new_features = train_eval.features.copy()
new_features["is_duplicate"] = Value('int32')
train_eval = train_eval.cast(new_features)

new_features = validation_eval.features.copy()
new_features["is_duplicate"] = Value('int32')
validation_eval = validation_eval.cast(new_features)

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')


# encode the training dataset in the form of sentences pair
encoded_train_eval = train_eval.map(lambda batch: tokenizer(batch['questions']['text'][0], batch['questions']['text'][1], 
                                                  padding='max_length', truncation=True, max_length=64))
encoded_train_eval.rename_column_("is_duplicate", "labels")

# encode the validation dataset in the form of sentences pair
encoded_validation_eval = validation_eval.map(lambda batch: tokenizer(batch['questions']['text'][0], batch['questions']['text'][1], 
                                                            padding='max_length', truncation=True, max_length=64))
encoded_validation_eval.rename_column_("is_duplicate", "labels")

Using custom data configuration default
Reusing dataset quora (C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)
Using custom data configuration default
Reusing dataset quora (C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)
Loading cached processed dataset at C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04\cache-58ce8a2d3d13bc51.arrow
Loading cached processed dataset at C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04\cache-077de9dbe62a5b12.arrow
Loading cached processed dataset at C:\Users\52673\.cache\huggingface\datasets\quora\default\0.0.0\36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04\cache-aec8ebd6b027502c.arrow
Loading cached processed dataset at C

In [16]:
# load the fine-tuned CL-BERT model
simcse_qqp = AutoModelForSequenceClassification.from_pretrained("simcse_qqp")

In [17]:
# load the metric module to compute accuracy
accuracy = load_metric("accuracy")

def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [18]:
# load the metric module to compute f1 score
f1 = load_metric("f1")

def compute_f1(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1.compute(predictions=predictions, references=labels)

## Accuracy

In [19]:
trainer_qqp = Trainer(
    model=simcse_qqp,    #use the fine-tuned model saved on
    train_dataset=encoded_train_eval,
    eval_dataset=encoded_validation_eval,
    compute_metrics=compute_accuracy,
)

trainer_qqp.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: questions.
***** Running Evaluation *****
  Num examples = 60644
  Batch size = 8
100%|██████████| 7581/7581 [08:48<00:00, 14.33it/s]


{'eval_loss': 0.6635891199111938,
 'eval_accuracy': 0.6436745597256117,
 'eval_runtime': 531.4313,
 'eval_samples_per_second': 114.114,
 'eval_steps_per_second': 14.265}

## F1 score

In [20]:
trainer_qqp = Trainer(
    model=simcse_qqp,    #use the fine-tuned model saved on
    train_dataset=encoded_train_eval,
    eval_dataset=encoded_validation_eval,
    compute_metrics=compute_f1,
)

trainer_qqp.evaluate() 

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: questions.
***** Running Evaluation *****
  Num examples = 60644
  Batch size = 8
100%|██████████| 7581/7581 [12:49<00:00,  9.85it/s]


{'eval_loss': 0.6635891199111938,
 'eval_f1': 0.0,
 'eval_runtime': 769.8367,
 'eval_samples_per_second': 78.775,
 'eval_steps_per_second': 9.848}